# **Overcoming limitations of Naive RAG with Enhanced Agentic Approach**

In this notebook, we'll explore some shortcomings of a naive RAG pipeline & build a more robust RAG system with an LLM agent that can utilize various email-tools and provide better output.

We'll be using [Unify](https://unify.ai/) for querying various Large Language models. Unify simplifies the process of navigating and selecting LLMs by providing a Single Sign On (SSO) feature, which allows access to various models with a single API key. It also features **Runtime Dynamic Routing**, which *automatically redirects requests to the optimal LLM provider based on user-defined constraints.*

We'll use [Qdrant](https://qdrant.tech/) for building the RAG engine. Qdrant is a vector similarity search engine that offers a user-friendly API to store, search, and manage vectors with an additional payload, supporting a wide range of applications including neural network or semantic-based matching, faceted search, and more

### **Installation**

Let's start by installing some neccessary packages!

In [37]:
#!pip install qdrant-client fastembed unifyai python-dotenv pandas numpy==1.26.4 dotenv

### **Preparing the environment**

To begin with, you need to obtain a **API key** for `Unify` as well as `Qdrant`:
 1. You can generate your `UNIFY_KEY` from the Unify's [console page](https://console.unify.ai/login?callbackUrl=%2F).
 2. You can get your `QDRANT_URL` and `QDRANT_API_KEY` from Qdrant's [SignUp Page](https://cloud.qdrant.io/login).

Once you have generated the keys, you should store them as [Colab secrets](https://medium.com/@parthdasawant/how-to-use-secrets-in-google-colab-450c38e3ec75) or if using locally in your **.env** file.

In [17]:
#Uncomment this when using google colab
#from google.colab import userdata
#QDRANT_URL = userdata.get('QDRANT_URL')
#QDRANT_KEY = userdata.get('QDRANT_API_KEY')
#UNIFY_KEY = userdata.get('UNIFY_KEY')

In [18]:
from dotenv import load_dotenv
load_dotenv()

True

In [24]:
import os
UNIFY_KEY=os.environ.get("UNIFY_KEY")
QDRANT_URL=os.environ.get("QDRANT_URL")
QDRANT_KEY=os.environ.get("QDRANT_API_KEY")

#### Let's instantiate our Unify & Qdrant Clients.


**Note** : We can use Qdrant in 3 different listed ways, we are using **Qdrant Cloud** in this workshop today.

In [25]:
#import qdrant_client
##Uncomment to initialise qdrant client in memory
#client = qdrant_client.QdrantClient(
#    location=":memory:",
#)

##Uncomment below to connect to Qdrant Cloud
#client = qdrant_client.QdrantClient(
#    QDRANT_URL,
#    api_key=QDRANT_KEY,
#)

## Uncomment below to connect to local Qdrant
#client = qdrant_client.QdrantClient("http://localhost:6333")

In [26]:
from qdrant_client import QdrantClient
from unify import Unify

unify_client = Unify(api_key=UNIFY_KEY, endpoint="llama-2-70b-chat@together-ai")
qdrant_client = QdrantClient(
    url=QDRANT_URL,
    api_key=QDRANT_KEY,
)

### **Building the source dataset**


We'll first craft a toy **email dataset** for John Doe, featuring exchanges with multiple entities. Our goal is to create a *compact yet diverse collection mirroring real-world email interactions*. Eventually, we'll use this data-set to build a **Q&A system** using RAG that John Doe can use *to ask questions about his emails*.

In [27]:
import datetime

**Case 1: AI Conference Invitation**

An email exchange between Jane Doe and John Doe where Jane initially agreed to attend an AI conference but later canceled due to unexpected work. This was in response to an invitation from John.

In [28]:
case_1 = [
    {
        'from_name': 'Jane Doe',
        'from_email': 'janedoe@example.com',
        'to_name': 'John Doe',
        'to_email': 'johndoe@example.com',
        'date': datetime.datetime(2024, 2, 27, 18, 49, 11, tzinfo=datetime.timezone.utc),
        "subject": "Re: AI Conference Invite",
        "body": "Hey John! Sorry, something urgent has come up. I won't be able to join."
    },

    {
        'from_name': 'Jane Doe',
        'from_email': 'janedoe@example.com',
        'to_name': 'John Doe',
        'to_email': 'johndoe@example.com',
        'date': datetime.datetime(2024, 2, 24, 19, 42, 11, tzinfo=datetime.timezone.utc),
        "subject": "Re: AI Conference Invite",
        "body": "Yeah, I think I should be able to join!"
    },

    {
        'from_name': 'Jane Doe',
        'from_email': 'janedoe@example.com',
        'to_name': 'John Doe',
        'to_email': 'johndoe@example.com',
        'date': datetime.datetime(2024, 2, 23, 19, 50, 11, tzinfo=datetime.timezone.utc),
        "subject": "Re: AI Conference Invite",
        "body": "Hmm, not sure, I'll get back to you tomorrow!"
    },

    {
        'from_name': 'John Doe',
        'from_email': 'johndoe@example.com',
        'to_name': 'Jane Doe',
        'to_email': 'janedoe@example.com',
        'date': datetime.datetime(2024, 2, 23, 19, 44, 11, tzinfo=datetime.timezone.utc),
        "subject": "AI Conference Invite",
        "body": "Hi Jane, do you think you would be able to join this conference futureofaiconference.com?"
    },
    {
        'from_name': "Future of AI",
        "from_email": "future@ai.com",
        "to_name": "John Doe",
        "to_email": "johndoe@example.com",
        'date': datetime.datetime(2024, 2, 23, 19, 42, 11, tzinfo=datetime.timezone.utc),
        "subject": "Best AI Conferences to Attend this Year!",
        "body": "Some of the best AI Conferences to look out for this year are awesomeAIconference and GlobalAIMeetup."
     }
]


**Case 2: Paper Reading Invitation Emails**

In these emails, John invited Jane Smith, Alice Johnson, and Bob Smith to present their respective papers in a reading group session. Bob Smith accepted the invitation to present his paper while rest of the authers didn't reply.

In [29]:
case_2 = [
{
    'from_name': 'John Doe',
    'from_email': 'johndoe@example.com',
    'to_name': 'Jane Smith',
    'to_email': 'janesmith@google.com',
    'date': datetime.datetime(2023, 4, 23, 19, 42, 11, tzinfo=datetime.timezone.utc),
    "subject": "Paper reading invitation",
    "body": """Hey Jane!
    My name is John, and I'm part of the engineering team on XYZ, I would love to ask you to present your paper 'Cool Agents stuff' on one of our reading group sessions!"""
},
{
    'from_name': 'John Doe',
    'from_email': 'johndoe@example.com',
    'to_name': 'Alice Johnson',
    'to_email': 'alicejohnson@microsoft.com',
    'date': datetime.datetime(2023, 4, 23, 19, 42, 11, tzinfo=datetime.timezone.utc),
    "subject": "Paper reading invitation",
    "body": """Hey Alice!
    My name is John, and I'm part of the engineering team on XYZ, I would love to ask you to present your paper 'Cool Rag stuff' on one of our reading group sessions!"""
},
{
   'from_name': 'Bob Smith',
    'from_email': 'bobsmith@example.com',
    'to_name': 'John Doe',
    'to_email': 'johndoe@example.com',
    'date': datetime.datetime(2023, 4, 24, 19, 42, 11, tzinfo=datetime.timezone.utc),
    "subject": "Re: Paper reading invitation",
    "body": """Hey John!
    I'd be happy to present!"""
},
{
    'from_name': 'John Doe',
    'from_email': 'johndoe@example.com',
    'to_name': 'Bob Smith',
    'to_email': 'bobsmith@unify.ai',
    'date': datetime.datetime(2023, 4, 23, 19, 42, 11, tzinfo=datetime.timezone.utc),
    "subject": "Paper reading invitation",
    "body": """Hey Bob!
    My name is John, and I'm part of the engineering team on XYZ, I would love to ask you to present your paper 'Cool prompt engineering stuff' on one of our reading group sessions!"""
}
]

**Case 3: Welcome & Follow Up Emails.**

An email exchange between Alex and John Doe, where Alex welcomes John Doe to NDS and expresses interest in learning how John found NDS to offer better support. Alex follows up when he doesn't receive a response. Later, the NDS Team sends John Doe an email announcing new features.

In [30]:
case_3 = [
    {
        'from_name': "NDS Team",
        "from_email": "noreply@nds.com",
        "to_name": "John Doe",
        "to_email": "johndoe@example.com",
        "subject": "New Features! Nexus Dynamics Solutions",
        'date': datetime.datetime(2024, 3, 23, 19, 42, 11, tzinfo=datetime.timezone.utc),
        "body": """We are happy to announce our new set of features! <lots of cool features>"""
    },
    {
        'from_name': "Alex",
        "from_email": "alex@nds.com",
        "to_name": "John Doe",
        "to_email": "johndoe@example.com",
        'date': datetime.datetime(2024, 3, 21, 20, 42, 11, tzinfo=datetime.timezone.utc),
        "subject": "Welcome to NDS!",
        "body": """Hey John, still no reply? is it not a suitable time?"""
    },
    {
        'from_name': "Alex",
        "from_email": "alex@nds.com",
        "to_name": "John Doe",
        "to_email": "johndoe@example.com",
        'date': datetime.datetime(2024, 3, 20, 19, 42, 11, tzinfo=datetime.timezone.utc),
        "subject": "Welcome to NDS!",
        "body": """Hey John,

        Great to see you signed up for NDS! Thought I'd introduce myself!

        I'd love to know more about how you came across NDS so that we can best support you.

        Would it be too difficult to connect in the coming weeks?"""

    }
]


### **Naive RAG**

#### Building a *knowledge store* using Qdrant

Let's build a knowledge base with John's emails using Qdrant.



This code defines a custom encoder `default_encoder` that converts datetime objects to ISO formatted strings. We will need *this* custom encoder while serializing the dictionaries to JSON strings.

In [31]:
import json

def default_encoder(obj):
    if isinstance(obj, datetime.datetime):
        return obj.isoformat()
    raise TypeError(f"Object of type {type(obj)} is not JSON serializable")


In [32]:
email_cases = [case_1, case_2, case_3]
email_strings = [json.dumps(email, default=default_encoder) for email in email_cases]

In [33]:
print(email_strings)

['[{"from_name": "Jane Doe", "from_email": "janedoe@example.com", "to_name": "John Doe", "to_email": "johndoe@example.com", "date": "2024-02-27T18:49:11+00:00", "subject": "Re: AI Conference Invite", "body": "Hey John! Sorry, something urgent has come up. I won\'t be able to join."}, {"from_name": "Jane Doe", "from_email": "janedoe@example.com", "to_name": "John Doe", "to_email": "johndoe@example.com", "date": "2024-02-24T19:42:11+00:00", "subject": "Re: AI Conference Invite", "body": "Yeah, I think I should be able to join!"}, {"from_name": "Jane Doe", "from_email": "janedoe@example.com", "to_name": "John Doe", "to_email": "johndoe@example.com", "date": "2024-02-23T19:50:11+00:00", "subject": "Re: AI Conference Invite", "body": "Hmm, not sure, I\'ll get back to you tomorrow!"}, {"from_name": "John Doe", "from_email": "johndoe@example.com", "to_name": "Jane Doe", "to_email": "janedoe@example.com", "date": "2024-02-23T19:44:11+00:00", "subject": "AI Conference Invite", "body": "Hi Jane,

In [34]:
import numpy as np
from typing import Tuple, List

def np_email_array(email_cases: List[List[str]]) -> Tuple[np.ndarray, np.ndarray]:
    """
    Convert a list of email threads into numpy arrays.

    Args:
        email_cases (List[List[str]]): A list of email threads, where each thread is a list of strings.

    Returns:
        Tuple[np.ndarray, np.ndarray]: A tuple containing two numpy arrays:
            - The first array contains individual emails.
            - The second array contains string representations of emails.
    """
    emails_array = []
    emails_array_str = []
    for thread in email_cases:
        for email in thread:
            emails_array_str.append(str(email))
            emails_array.append(email)
    return np.array(emails_array), np.array(emails_array_str)

emails_array, emails_array_str = np_email_array(email_cases)
print(emails_array)

[{'from_name': 'Jane Doe', 'from_email': 'janedoe@example.com', 'to_name': 'John Doe', 'to_email': 'johndoe@example.com', 'date': datetime.datetime(2024, 2, 27, 18, 49, 11, tzinfo=datetime.timezone.utc), 'subject': 'Re: AI Conference Invite', 'body': "Hey John! Sorry, something urgent has come up. I won't be able to join."}
 {'from_name': 'Jane Doe', 'from_email': 'janedoe@example.com', 'to_name': 'John Doe', 'to_email': 'johndoe@example.com', 'date': datetime.datetime(2024, 2, 24, 19, 42, 11, tzinfo=datetime.timezone.utc), 'subject': 'Re: AI Conference Invite', 'body': 'Yeah, I think I should be able to join!'}
 {'from_name': 'Jane Doe', 'from_email': 'janedoe@example.com', 'to_name': 'John Doe', 'to_email': 'johndoe@example.com', 'date': datetime.datetime(2024, 2, 23, 19, 50, 11, tzinfo=datetime.timezone.utc), 'subject': 'Re: AI Conference Invite', 'body': "Hmm, not sure, I'll get back to you tomorrow!"}
 {'from_name': 'John Doe', 'from_email': 'johndoe@example.com', 'to_name': 'Jane

Let's extract the content and metadeta from the documents.
We need these to create our [Qdrant collections](https://qdrant.tech/documentation/concepts/collections/) which is a fundamental way to organize data, consisting of named sets of vectors with payloads that you can search through.

To help us provide seamless embedding creations throughout the workshop we are using [Fastembed](https://qdrant.github.io/fastembed/)


In [38]:
## To look at the complete list of supported models
from fastembed.embedding import TextEmbedding
import pandas as pd
pd.DataFrame(TextEmbedding.list_supported_models())

,model,dim,description,size_in_GB,sources,model_file,additional_files
0,BAAI/bge-base-en,768,Base English model,0.420,{'url': 'https://storage.googleapis.com/qdrant...,model_optimized.onnx,NaN
1,BAAI/bge-base-en-v1.5,768,"Base English model, v1.5",0.210,{'url': 'https://storage.googleapis.com/qdrant...,model_optimized.onnx,NaN
2,BAAI/bge-large-en-v1.5,1024,"Large English model, v1.5",1.200,{'hf': 'qdrant/bge-large-en-v1.5-onnx'},model.onnx,NaN
3,BAAI/bge-small-en,384,Fast English model,0.130,{'url': 'https://storage.googleapis.com/qdrant...,model_optimized.onnx,NaN
4,BAAI/bge-small-en-v1.5,384,Fast and Default English model,0.067,{'hf': 'qdrant/bge-small-en-v1.5-onnx-q'},model_optimized.onnx,NaN
5,BAAI/bge-small-zh-v1.5,512,Fast and recommended Chinese model,0.090,{'url': 'https://storage.googleapis.com/qdrant...,model_optimized.onnx,NaN
6,sentence-transformers/all-MiniLM-L6-v2,384,"Sentence Transformer model, MiniLM-L6-v2",0.090,{'url': 'https://storage.googleapis.com/qdrant...,model.onnx,NaN
7,sentence-transformers/paraphrase-multilingual-...,384,"Sentence Transformer model, paraphrase-multili...",0.220,{'hf': 'qdrant/paraphrase-multilingual-MiniLM-...,model_optimized.onnx,NaN
8,nomic-ai/nomic-embed-text-v1,768,8192 context length english model,0.520,{'hf': 'nomic-ai/nomic-embed-text-v1'},onnx/model.onnx,NaN
9,nomic-ai/nomic-embed-text-v1.5,768,8192 context length english model,0.520,{'hf': 'nomic-ai/nomic-embed-text-v1.5'},onnx/model.onnx,NaN


We use the *DEFAULT_EMBEDDING_MODEL* for this workshop i.e. *BAAI/bge-small-en* however should you want to experiment with another embedding model :

In [39]:
#qdrant_client.DEFAULT_EMBEDDING_MODEL
## For custom model supported by Fastembed
#embedding_model = TextEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2", max_length=384)
#qdrant_client.set_model(embedding_model_name=embedding_model_name)

In [ ]:
qdrant_client.add(collection_name="email", documents=emails_array_str, )

['f73c7b209a4f4a8a8c2d89ea1d75cf1c',
 '4e030fafbb2547d6af0cc1d8967aa335',
 '8feaa379f2f7496cbf07e74907457c29',
 '007705a81032453dbcab08dabd2d7b04',
 'a5ef6afd546f45438fef83c2014b55cc',
 '1964b7baba9940c18d0d9dc1604371c3',
 '98010e905fda439d9f5f8921c5173b5b',
 '893edb0b76a844e198d6c34dd83a83f7',
 'f92c7d8b0eeb4a04be006456b76984d0',
 '280f9b1c1e964a958676f4c71a5d8c7c',
 '47a92f85bf4c42c5b0ebb1c2c9e30124',
 'e3b0a7cdd9f54f0988c548ce8621f57b']

#### Retrieval Augmented Generatation

Now, let's define a function which will retreive the relevant context from our Qdrant collection based on the user prompt and append it to the query.

In [40]:
def add_context(query, client, collection_name, limit):
    search_result = client.query(collection_name=collection_name, query_text=query, limit=limit)

    context = "\n".join(r.document for r in search_result)
    #print(context)
    system_prompt = """You're assisting John Doe, who has a query based on his emails.
          Your objective is to deliver a response that is clear and concise, addressing his question while referencing pertinent information from his email threads.
          Please keep in mind:
          - Fully comprehend the question.
          - For general queries (e.g., "hi," "good morning"), respond normally without any contextual references.
          - For specific queries related to email content, extract relevant information from the provided context.
          - Formulate a response that directly answers the query, supported by accurate information from the relevant source.
          - Maintain a friendly and professional tone throughout your response.
          - If the answer cannot be found within the provided context, honestly state: "I wasn't able to find any such information from the provided context."
          """

    prompt_end = (f"\n\nQuestion: {query}\nAnswer:")

    user_prompt = f"Context: {context}" + prompt_end
    return system_prompt, user_prompt


In [41]:
def query_with_context(query, client, collection_name, retrieval_window_size):
    system_prompt, user_prompt = add_context(query, client, collection_name, retrieval_window_size)
    query_len = len(system_prompt) + len(user_prompt)
    credits_before = unify_client.get_credit_balance()
    response = unify_client.generate(
        system_prompt=system_prompt,
        user_prompt=user_prompt,
    )
    credits_after = unify_client.get_credit_balance()
    print("------------------------")
    print("System's Response")
    print("------------------------")
    print(response)
    print()
    print("------------------------")
    print(f"The length of the prompt is {query_len}")
    print("------------------------")
    print(f"Credits used: {credits_before - credits_after}")
    print("------------------------")

Alright, let's now ask some questions about John's emails.

In [42]:
retrieval_window_size = 2
query_with_context("Can you summarize the last few emails I recieved?",
                   qdrant_client,
                   "email",
                   retrieval_window_size)

100%|████████████████████████████████████████████████████████████████████████| 77.7M/77.7M [00:12<00:00, 6.25MiB/s]
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


------------------------
System's Response
------------------------
Sure, I can help you with that! Based on the email threads you provided, it appears that Jane Doe invited you to an AI conference and you replied asking for more information. Later, Jane apologized for the confusion and mentioned that something urgent had come up.

In summary, the last few emails you received were:

1. An invitation to an AI conference from Jane Doe
2. Your response asking for more information
3. An apology from Jane Doe for the confusion and a mention of an urgent matter

I hope this summary helps! If you have any further questions or concerns, please don't hesitate to ask.

------------------------
The length of the prompt is 1356
------------------------
Credits used: 0.00047430000000048267
------------------------


Clearly this **response is incorrect and unsatisfactory**.
The latest message in John Doe's email is dated 23 March, 2024 from Nexus Dynamic Solutions. The reason why RAG fails here is because it relies on a *naive semantic matching* of the user query with the documents stored in it's database. Since the user query here doesn't explicitly mention any dates, a set of k ( retrieval window size) random documents will be retrieved by the engine.
Let's increase the retrieval window size and see what happens ...

In [43]:
retrieval_window_size = 5
query_with_context("Can you summarize the last few emails I recieved?",
                   qdrant_client,
                   "email",
                   retrieval_window_size)

------------------------
System's Response
------------------------
Sure, I can help you with that! Based on the provided context, it appears that you have received a few emails related to various topics. Here's a brief summary of the last few emails you received:

1. From Jane Doe (janedoe@example.com) on September 27, 2023, with the subject "Re: AI Conference Invite": Jane apologizes for not being able to make it to the AI conference and mentions that something urgent has come up.
2. From John Doe (johndoe@example.com) on March 21, 2024, with the subject "Welcome to NDS!": John sends an email to Jane Smith (janesmith@google.com) and copied you, welcoming you to NDS. He asks if it's not a suitable time for you to reply.
3. From Alex (alex@nds.com) on April 24, 2024, with the subject "Re: Paper reading invitation": Alex invites you to present a paper and mentions that he'd be happy to do it.
4. From John Doe (johndoe@example.com) on September 23, 2023, with the subject "Re: AI Conferen

The system still isn't able to find the correct email. If we increase the window size to 12 ( which means the system will retrieve all the documents), the LLM should be able to find the correct email. Let's explore this!

In [44]:
retrieval_window_size = 12
query_with_context("Can you summarize the last few emails I recieved?",
                   qdrant_client,
                   "email",
                   retrieval_window_size)

------------------------
System's Response
------------------------
Sure, I can summarize the last few emails you received. It appears that you received an invitation to a conference from the NDS Team, which you forwarded to Jane Doe. Jane apologized for not being able to join the conference and suggested that you might be interested in attending. You replied that you would get back to her tomorrow regarding your availability. You also received an email from Alex, who expressed interest in presenting a paper at the conference. Finally, you received an email from the Future of AI, stating that they think they should be able to join the conference.

Here's a summary of the last few emails you received:

* NDS Team invited you to a conference.
* You forwarded the invitation to Jane Doe.
* Jane Doe apologized for not being able to join and suggested that you might be interested.
* You replied that you would get back to her tomorrow regarding your availability.
* Alex expressed interest in 

As expected, the system does return the correct answer now.
However, **at the cost of much higher input tokens**! It's also missing the dates now. Let's try another LLM!


In [45]:
unify_client.set_endpoint("llama-3-70b-chat@together-ai")

In [46]:
retrieval_window_size = 12
query_with_context("Can you summarize the last few emails I recieved?",
                   qdrant_client,
                   "email",
                   retrieval_window_size)

------------------------
System's Response
------------------------
Hi John,

Based on the provided email threads, here's a summary of the last few emails you received:

1. On 2024-04-24, Alex from NDS replied to your email about a paper reading invitation, mentioning they'd be happy to present.
2. On 2024-03-21, you sent an email to Jane Smith, asking if it wasn't a suitable time, but there was no reply mentioned.
3. On 2023-09-27, Jane Doe replied to your email about the AI Conference Invite, apologizing for something urgent that came up.

These are the most recent email exchanges I could find in the provided context. Let me know if you need any further assistance!

------------------------
The length of the prompt is 2952
------------------------
Credits used: 0.0009143999999992047
------------------------


Another lesson here is that the final response quality also depends on the summarizing LLM.

Let's try some more examples!

In [47]:
retrieval_window_size = 5
query_with_context("Hello! Will Jane be able to join the AI conference?",
                   qdrant_client,
                   "email",
                   retrieval_window_size)

------------------------
System's Response
------------------------
Hi John!

According to your email thread, Jane initially responded positively to the conference invite, saying "Yeah, I think I should be able to join!" However, in a later email, she mentioned that something urgent has come up and apologized, which might indicate that she may not be able to join after all.

So, to answer your question, it seems uncertain whether Jane will be able to join the AI conference.

------------------------
The length of the prompt is 2236
------------------------
Credits used: 0.0006218999999987318
------------------------


Well, that's **incorrect** again.
We know that Jane later sent another email stating that *she won't be able to join the conference*.
If we increase the retreival window size, the system will have access to more email exchanges between John & Jane and should be able to return the correct answer.
Let's explore this.

In [48]:
retrieval_window_size = 5
query_with_context("Hello! Will Jane be able to join the AI conference?",
                   qdrant_client,
                   "email",
                   retrieval_window_size)

------------------------
System's Response
------------------------
Hi John!

According to the email thread, Jane initially responded positively to the AI Conference Invite on September 23, saying "Yeah, I think I should be able to join!" However, later on September 27, she sent another email apologizing and stating that something urgent has come up, implying that she might not be able to attend the conference after all.

So, to answer your question, it's unclear if Jane will be able to join the AI conference.

------------------------
The length of the prompt is 2236
------------------------
Credits used: 0.0006309000000044307
------------------------


As expected, the system does **correctly answer** the question now.

The lesson here is that there's **no one-size-fits-all** value for the retrieval window size & while one does increase the accuracy of the system by choosing a large value but it comes with an increase in LLM Usage cost.

Let's try another example with a different LLM now!

In [49]:
unify_client.set_endpoint("gemma-7b-it@anyscale")

In [50]:
retrieval_window_size = 2
query_with_context("Hello! What was the name of the guy from Nexus Dynamic Solutions that sent me a couple of emails?",
                   qdrant_client,
                   "email",
                   retrieval_window_size)

------------------------
System's Response
------------------------
**Answer:**

The provided text does not mention the name of the guy from Nexus Dynamic Solutions who sent emails to you, therefore I cannot answer this question.

I wasn't able to find any information about the name of the guy from Nexus Dynamic Solutions that sent emails to you within the provided context.

------------------------
The length of the prompt is 1514
------------------------
Credits used: 6.88499999981218e-05
------------------------


In [51]:
retrieval_window_size = 4
query_with_context("Hello! What was the name of the guy from Nexus Dynamic Solutions that sent me a couple of emails?",
                   qdrant_client,
                   "email",
                   retrieval_window_size)

------------------------
System's Response
------------------------
Hello, John Doe,

Based on the provided context, I understand your query about the name of the guy from Nexus Dynamic Solutions who sent you a couple of emails.

In the email thread, the sender's name is Alex. Therefore, the answer to your question is Alex.

Please let me know if you have any further questions or require further assistance.

Sincerely,
[Your Name]

------------------------
The length of the prompt is 2013
------------------------
Credits used: 0.0001030499999998824
------------------------


In [52]:
retrieval_window_size = 6
query_with_context("Hello! What was the name of the guy from Nexus Dynamic Solutions that sent me a couple of emails?",
                   qdrant_client,
                   "email",
                   retrieval_window_size)

------------------------
System's Response
------------------------
Hello, John Doe.

Based on the provided context, it appears that the email threads you provided do not contain any information about the name of the guy from Nexus Dynamic Solutions who sent you a couple of emails. Therefore, I unfortunately cannot answer your query.

Please provide more information or context if available, and I will be happy to assist you further.

------------------------
The length of the prompt is 2525
------------------------
Credits used: 0.0001278000000013435
------------------------


In [53]:
retrieval_window_size = 8
query_with_context("Hello! What was the name of the guy from Nexus Dynamic Solutions that sent me a couple of emails?",
                   qdrant_client,
                   "email",
                   retrieval_window_size)

------------------------
System's Response
------------------------
**Answer:**

The text does not identify the name of the guy from Nexus Dynamic Solutions who sent the emails, therefore I cannot answer this query.

------------------------
The length of the prompt is 2756
------------------------
Credits used: 0.00013500000000021828
------------------------


In [54]:
retrieval_window_size = 12
query_with_context("Hello! What was the name of the guy from Nexus Dynamic Solutions that sent me a couple of emails?",
                   qdrant_client,
                   "email",
                   retrieval_window_size)

------------------------
System's Response
------------------------
**Answer:**

The text does not provide any information about the name of the guy from Nexus Dynamic Solutions who sent you emails, therefore I cannot answer this question.

------------------------
The length of the prompt is 3765
------------------------
Credits used: 0.00019409999999453476
------------------------


Maybe our summarizing LLM is too weak, let's try another one!

In [55]:
unify_client.set_endpoint("gpt-4o@openai")

In [56]:
retrieval_window_size = 4
query_with_context("Hello! What was the name of the guy from Nexus Dynamic Solutions that sent me a couple of emails?",
                   qdrant_client,
                   "email",
                   retrieval_window_size)

------------------------
System's Response
------------------------
Hello! The name of the guy from Nexus Dynamics Solutions who sent you a couple of emails is Alex.

------------------------
The length of the prompt is 2013
------------------------
Credits used: 0.002965000000003215
------------------------


Aha, GPT-4o is able to get the right answer!

**Observation of limitations of Naive RAG**


 Let's note down some of the limitations of the **Naive RAG** system that we observed above:

1. **Incomplete Information Retrieval**:

  * **Issue**: Documents may **lack necessary information** to properly answer the user query.
  * **Challenge**: Determining **optimal retrieval window size**, impacting LLM API costs.

2. **Vulnerability to Vague Queries**:

  * **Issue**: Naive LLM **struggles with ambiguous/polysemus user queries**.
  * **Reason**: Direct query-document matching without robust contextual understanding.

3. **Varied Response Quality:**

  * **Issue**: Response quality **influenced by LLM** summarization capabilities.
  * **Variation**: Some LLMs draw better conclusions from provided context than others.



# **Agentic RAG**

In this section, we aim to tackle the limitations of the Naive RAG system with a **smarter LLM agent** approach.
This enhanced approach will leverage a spectrum of email tools which we will implement, empowering the agent to respond accurately to inquiries.  
We'll explain each part of our agent one step at a time to make it easy to follow.

### **Utilities**

This section contains some useful utility methods that will need for the sections below.

In [57]:
from typing import Set, Dict

def get_all_domains(email_cases: List[List[Dict[str, str]]]) -> Set[str]:
    """
    Extract unique email domains from a list of email threads.

    Args:
        email_cases (List[List[Dict[str, str]]]): A list of email threads, where each thread is a list of dictionaries representing emails.

    Returns:
        Set[str]: A set containing unique domains extracted from the emails.
    """
    domains = set()
    for thread in email_cases:
        for email in thread:
            if email["to_email"]:
                domains.add(email["to_email"].split("@")[1])
            if email["from_email"]:
                domains.add(email["from_email"].split("@")[1])
    return domains

domains = get_all_domains(email_cases)
print(domains)


{'nds.com', 'microsoft.com', 'ai.com', 'example.com', 'google.com', 'unify.ai'}


In [58]:
def get_all_emails(email_cases: List[List[Dict[str, str]]]) -> Set[str]:
    """
    Extract unique email addresses from a list of email threads.

    Args:
        email_cases (List[List[Dict[str, str]]]): A list of email threads, where each thread is a list of dictionaries representing emails.

    Returns:
        Set[str]: A set containing unique email addresses extracted from the emails.
    """
    all_emails = set()
    for thread in email_cases:
        for email in thread:
            if email["to_email"]:
                all_emails.add(email["to_email"])
            if email["from_email"]:
                all_emails.add(email["from_email"])
    return all_emails


all_emails = get_all_emails(email_cases)
print(all_emails)


{'noreply@nds.com', 'janedoe@example.com', 'bobsmith@unify.ai', 'janesmith@google.com', 'alex@nds.com', 'alicejohnson@microsoft.com', 'johndoe@example.com', 'bobsmith@example.com', 'future@ai.com'}


### **Tools**

This section contains some more useful functions that our agent can use to address any user query.

In [59]:
def search_query(query=None, subject=None, limit = 5):
    '''Useful for fetching a bunch of emails that might be relevant to the query or email subject.
    Make sure to include important keywords in the query to make good use of this tool.
    The tool does not retrieve all relevant emails as it relies on fuzzy matching and semantic search, and should usually be used in combination with other tools.
    The tool returns emails in no specific order, with no regard to its order in the original conversation, you must make sure you do not rely on it to make any final conclusions!
    This tool does not support email

    Parameters:
    - query (str): A natural language, question-like query that will be used to find relevant emails, e.g "name of person working at X". Defaults to None
    - subject (str): An email subject that can be used to retrieve other emails with similiar subject. Defaults to None
    - limit (int): The number of emails retrieved during the search. Defaults to 5.

    One of query or subject must be provided.
  The search tool DOES NOT support gmail search operators such as "from:", "label:", etc, instead use it as it is documented above.


    Returns:
    A bunch of truncated relevant emails.
    '''

    emails = set()
    if query:
        search_result = qdrant_client.query(collection_name="email", query_text=query, limit=limit)
        return "\n".join(r.document for r in search_result)
        #q_emb = model.encode(query, normalize_embeddings=True)
        #sim = q_emb @ name_email_subject_body_embs.T
        #emails.update(emails_array_str[sim.argsort()[-15:]].tolist())
    if subject:
        idx = [i for i, email in enumerate(emails_array) if subject.lower() in email['subject'].lower()]
        #print(idx)
        emails.update(emails_array_str[idx])
    return "\n\n".join(list(emails))

In [60]:
print(search_query(subject ="AI conference"))

{'from_name': 'Jane Doe', 'from_email': 'janedoe@example.com', 'to_name': 'John Doe', 'to_email': 'johndoe@example.com', 'date': datetime.datetime(2024, 2, 27, 18, 49, 11, tzinfo=datetime.timezone.utc), 'subject': 'Re: AI Conference Invite', 'body': "Hey John! Sorry, something urgent has come up. I won't be able to join."}

{'from_name': 'Future of AI', 'from_email': 'future@ai.com', 'to_name': 'John Doe', 'to_email': 'johndoe@example.com', 'date': datetime.datetime(2024, 2, 23, 19, 42, 11, tzinfo=datetime.timezone.utc), 'subject': 'Best AI Conferences to Attend this Year!', 'body': 'Some of the best AI Conferences to look out for this year are awesomeAIconference and GlobalAIMeetup.'}

{'from_name': 'John Doe', 'from_email': 'johndoe@example.com', 'to_name': 'Jane Doe', 'to_email': 'janedoe@example.com', 'date': datetime.datetime(2024, 2, 23, 19, 44, 11, tzinfo=datetime.timezone.utc), 'subject': 'AI Conference Invite', 'body': 'Hi Jane, do you think you would be able to join this conf

In [61]:
print(search_query("Paper Reading Session"))

paper 'Cool prompt engineering stuff' on one of our reading group sessions!"}]
your paper 'Cool Rag stuff' on one of our reading group sessions!"}, {"from_name": "Bob Smith", "from_email": "bobsmith@example
paper 'Cool Agents stuff' on one of our reading group sessions!"}, {"from_name": "John Doe", "from_email": "johndoe@example
.com", "date": "2024-04-23T19:42:11+00:00", "subject": "Paper reading invitation", "body": "Hey Alice!\n    My name is John, and I'm part of the engineering team on XYZ, I would love to ask you to present your paper 'Cool Rag stuff' on one of our reading
.com", "date": "2024-04-23T19:42:11+00:00", "subject": "Paper reading invitation", "body": "Hey Jane!\n    My name is John, and I'm part of the engineering team on XYZ, I would love to ask you to present your paper 'Cool Agents stuff' on one of our reading


In [62]:
from ast import literal_eval

In [63]:
def find_email_addresses_by_entity(entity_name):
    '''Useful for fetching emails addressess associated with an entity, can be used to know if anyone from entity has emailed the client.

    Parameters:
    - entity_name (str): Name of entity such e.g Google, Meta, MIT.

    Returns:
    List of potential email addresses associated with entity that might have interacted with the client.
    '''
    #llm = Unify(model="llama-3-70b-chat@together-ai", temperature=0.1, api_key=UNIFY_KEY)
    '''Useful for searching for email the client has interacted with based on the company they work at.'''
    prompt = f"""Which of the following email domains is most likely associated with {entity_name}, return your answer as a list. Do not output anything else.
    ```
    # Example output
    list: ["domain1.com", "domain2.ai"]
    ```
    {domains}"""
    domains_list = unify_client.generate(user_prompt = prompt).strip("list:")
    #print(domains_list)
    #p = ChatMessage(role=MessageRole.USER, content=prompt)
    #a = ChatMessage(role=MessageRole.ASSISTANT, content="list: ")
    #domains_list = llm.chat([p, a]).message.content.strip("list:")
    domains_list = literal_eval(domains_list.strip())
    potential_emails = set()
    for email in all_emails:
        if email.split("@")[1].strip() in domains_list: #and not email.split("@")[1].strip().endswith("bounces.google.com"):
            potential_emails.add(email)
    return "Potential emails:\n" + "\n".join(potential_emails)

In [64]:
print(find_email_addresses_by_entity("Nexus Dynamic Solutions"))

Potential emails:
noreply@nds.com
alex@nds.com


In [65]:
def find_conversations(email_1, email_2, subject=None):
    '''Useful for fetching full conversations between two email addresses to get the full picture or extra information.
    This tool should be used to get the full context is needed for the task, it is one of the most useful tools.

    Parameters:
    - email_1 (str): first email address.
    - email_2 (str): second email address.
    - subject (str): Optional email subject to only return the conversation/thread with that subject. Defaults to None

    Returns:
    Full Conversations between two email addresses in chronological order.
    '''
    gr = []
    for email in emails_array:
        if (
            email_1 in email["to_email"] and email_2 in email["from_email"]
        or email_2 in email["to_email"] and email_1 in email["from_email"]
        ):
            if subject is None:
                gr.append(email)
            else:
                if subject.lower().strip() in email["subject"].lower().strip():
                    gr.append(email)
    return "\n\n".join([str(x) for x in sorted(gr, key=lambda x: x["date"])][:10])

In [66]:
print(find_conversations("johndoe@example.com", "janedoe@example.com"))

{'from_name': 'John Doe', 'from_email': 'johndoe@example.com', 'to_name': 'Jane Doe', 'to_email': 'janedoe@example.com', 'date': datetime.datetime(2024, 2, 23, 19, 44, 11, tzinfo=datetime.timezone.utc), 'subject': 'AI Conference Invite', 'body': 'Hi Jane, do you think you would be able to join this conference futureofaiconference.com?'}

{'from_name': 'Jane Doe', 'from_email': 'janedoe@example.com', 'to_name': 'John Doe', 'to_email': 'johndoe@example.com', 'date': datetime.datetime(2024, 2, 23, 19, 50, 11, tzinfo=datetime.timezone.utc), 'subject': 'Re: AI Conference Invite', 'body': "Hmm, not sure, I'll get back to you tomorrow!"}

{'from_name': 'Jane Doe', 'from_email': 'janedoe@example.com', 'to_name': 'John Doe', 'to_email': 'johndoe@example.com', 'date': datetime.datetime(2024, 2, 24, 19, 42, 11, tzinfo=datetime.timezone.utc), 'subject': 'Re: AI Conference Invite', 'body': 'Yeah, I think I should be able to join!'}

{'from_name': 'Jane Doe', 'from_email': 'janedoe@example.com', 'to

In [67]:
def get_emails(num_emails):
      '''Useful for fetching the latest emails.
      Parameters:
      - num_emails (int): Number of latest emails to fetch.

      Returns:
      Latest `num_emails` recieved, emails are truncated if too long.
      '''
      return "\n\n".join([str(x) for x in sorted(emails_array, key=lambda x: x["date"], reverse=True)[:num_emails]])

In [68]:
print(get_emails(3))

{'from_name': 'NDS Team', 'from_email': 'noreply@nds.com', 'to_name': 'John Doe', 'to_email': 'johndoe@example.com', 'subject': 'New Features! Nexus Dynamics Solutions', 'date': datetime.datetime(2024, 3, 23, 19, 42, 11, tzinfo=datetime.timezone.utc), 'body': 'We are happy to announce our new set of features! <lots of cool features>'}

{'from_name': 'Alex', 'from_email': 'alex@nds.com', 'to_name': 'John Doe', 'to_email': 'johndoe@example.com', 'date': datetime.datetime(2024, 3, 21, 20, 42, 11, tzinfo=datetime.timezone.utc), 'subject': 'Welcome to NDS!', 'body': 'Hey John, still no reply? is it not a suitable time?'}

{'from_name': 'Alex', 'from_email': 'alex@nds.com', 'to_name': 'John Doe', 'to_email': 'johndoe@example.com', 'date': datetime.datetime(2024, 3, 20, 19, 42, 11, tzinfo=datetime.timezone.utc), 'subject': 'Welcome to NDS!', 'body': "Hey John,\n\n        Great to see you signed up for NDS! Thought I'd introduce myself!\n\n        I'd love to know more about how you came acros

In [69]:
tools = [find_conversations, find_email_addresses_by_entity, get_emails, search_query]

In [70]:
tool_format = '''> Tool Name: {tool_name}
Tool Description: {tool_desc}'''

In [71]:
formatted_tools = "\n".join(tool_format.format(tool_name=t.__name__, tool_desc=t.__doc__)
for t in tools)

### **Prompts**

This section contains the **system prompt** that gives the LLM instructions on how to answer the user query using the tools that we defined above.

In [72]:
sys_prompt  = """You're assisting John Doe with the email address: johndoe@example.com, who has a query based on his emails.
          Your objective is to deliver a response that is clear and concise,
         addressing his question while referencing pertinent information from his email threads.


Tools
You have access to a wide variety of tools to help you navigate his mailbox.
You are responsible for using the tools in any sequence you deem appropriate to complete the task at hand.
This may require breaking the task into subtasks and using different tools to complete each subtask.

You have access to the following tools:
{formatted_tools}


Output Format
Please answer in the same language as the question and use the following format:

Thought: Your thought process on how to tackle the question and which tool to use.
Action: tool name (ONLY one of {tools_list}).
Action Input: the input to the tool, in a JSON format representing the kwargs (e.g. {{"input": "hello world", "num_beams": 5}})
Please ALWAYS start with a Thought.

Please use a valid JSON format for the Action Input. Do NOT do this {{'input': 'hello world', 'num_beams': 5}}.

If this format is used, the user will respond in the following format:

Observation: tool response

You should keep repeating the above format till you have enough information to answer the question without using any more tools.
At that point, you MUST respond in the one of the following two formats:

Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: [your answer here]
Thought: I cannot answer the question with the provided tools.
Answer: [your answer here]

Important tips you must follow:
- Emails can be about various topics ranging from meetings, to newsletters, to other types of content, do not make premature assumptions about the question if its not clear.
- Do not rely on expanding emails to get the full conversation between the client and other people if you need the full conversation between them, rather use the appropraite tool for that.
- Remember that the `search` tool will only retrieve a subset of the potentially relevant emails, you will most likely need to use more tools alongside it to get the full picture.
- You can consider using the tool responsible for searching to also search for more specific email, by passing a shared email subject, this might retrieve even more relevant emails.
- You should never make a conclusion based on a single email, you must always retrieve the full conversation to get the full picture!
- When you are uncertain about the information in a single email, the tool responsible for fetching the entire conversation will give you the context you need!
- Never assume a hypothetical email domain when using any of the tools, as this will almost always lead to wrong and confusing results.
- Try to do your best to answer the user's query instead of prematurely returning a final answer, as this provides a much better user experience to them, some tasks might require the usage of many tools and going through many emails, do not return your final response until you have exhausted your options!

Current Conversation
Below is the current user query.
"""

In [73]:
sys_prompt = sys_prompt.format(formatted_tools=formatted_tools,
                               tools_list=[t.__name__ for t in tools])

### **Agentic RAG examples**

In [74]:
unify_client.set_endpoint("claude-3-opus@anthropic")

In [75]:
import re

In [76]:
def generate_response(prompt: str) -> str:
    """
    Generate a response to a given prompt by making use of differernt email tools.

     The generation process involves multiple reasoning steps, which include extracting thoughts, actions, and observations
     from the generated responses. The function iterates through these steps to refine the response
     to ultimately answer the user's question.

    Args:
        prompt (str): The prompt provided to generate the response.

    Returns:
        str: The generated response.

    Raises:
        Exception: If the tool action is not implemented.
    """
    messages = []
    response = unify_client.generate(user_prompt=prompt, system_prompt=sys_prompt)
    messages.append({"role":"system", "content": sys_prompt})
    messages.append({"role": "user", "content": prompt })
    # Just do a maximum of 10 reasoning steps
    for i in range(10):
        try:
            response = re.search("(.*?)(?=Observation:)", response.strip(), re.DOTALL|re.MULTILINE).group(0)
        except AttributeError:
            pass

        t = re.search("Thought: (.*)", response).group(1)
        print(f"Thought: {t}...")
        #print("------------------")
        #print(response)

        if re.search("Action: (.*)", response) is not None:
            action = re.search("Action: (.*)", response).group(1).strip()
            action_input = re.search("Action Input: (\{.*?\})", response, flags=re.DOTALL).group(1)
        elif re.search("Answer: (.*)", response) is not None:
            break

        ips = json.loads(action_input)
        if action == "search_query":
            if "query" in ips:
                output = search_query(query=ips["query"])
            else:
                output = search_query(subject=ips["subject"])

        elif action == "find_conversations":
            if "subject" in ips:
                output = find_conversations(email_1=ips["email_1"], email_2=ips["email_2"], subject=ips["subject"])
            else:
                output = find_conversations(email_1=ips["email_1"], email_2=ips["email_2"])

        elif action == "find_email_addresses_by_entity":
            output = find_email_addresses_by_entity(entity_name=ips["entity_name"])

        elif action == "get_emails":
            output = get_emails(num_emails=ips["num_emails"])
        else:
            raise Exception("Tool is not implemented", action)

        tool_observation = f'''Observation:
        {output}'''
        messages.append({'role': 'assistant', "content": response})
        messages.append({'role': 'user', 'content': tool_observation})
        response = unify_client.generate(messages=messages, system_prompt=sys_prompt)

    answer = re.search("Answer: (.*)", response, flags=re.DOTALL).group(1)
    print("Answer: ", answer)
    #return answer


In [77]:
prompt = "Can you summarize the last few emails I recieved?"
generate_response(prompt)

Thought: To summarize the latest emails, I will use the `get_emails` tool to retrieve the 5 most recent emails. This should give me a good overview of the client's recent email activity....
Thought: The 5 latest emails show a range of topics. There are 3 emails related to NDS, which seems to be a new service John signed up for. Alex from NDS reached out twice to connect with John, but it's unclear if John has responded yet. There's also an email about new NDS features being announced....
Answer:  Based on the 5 most recent emails:

- You signed up for a service called NDS (Nexus Dynamics Solutions). Alex from NDS has reached out twice to connect with you and learn more about how you found their service, but it's unclear if you've replied yet. NDS also sent an announcement email about new features being released.

- You had an email exchange with Jane Doe regarding an AI conference invite. She initially said she could attend, but in her latest email had to cancel due to an urgent issue 

In [78]:
prompt = "Hey! Will Jane be able to attend the AI conference?"
generate_response(prompt)

Thought: To determine if Jane will be able to attend the AI conference, I should search for emails related to Jane and the conference. The search_query tool can help find relevant emails....
Thought: The search results provide some relevant information about Jane's attendance at the AI conference, but the emails are not in chronological order and there seems to be a conversation between Jane and John that is not fully captured. To get a clearer picture, I should use the find_conversations tool to retrieve the full thread between Jane and John about the AI conference....
Thought: Based on the full conversation between Jane and John about the AI conference, it appears that:...
Answer:  No, based on the email conversation, Jane will not be able to attend the AI conference. Although she initially thought she could join when John invited her, she later informed him that due to an urgent matter that came up, she won't be able to attend the conference.


In [79]:
prompt = "Who are the researchers (and their papers) that accepted and declined my Paper reading invitation?"
generate_response(prompt)

Thought: To answer this query, I will need to find emails related to the paper reading invitation and see which researchers accepted or declined the invitation. I should start by searching for relevant emails using keywords from the query....
Thought: The search results provide some relevant information about the paper reading invitations sent out and the responses. However, to get a clear picture of who accepted and declined, I will need to find the full conversation threads for each researcher invited....
Thought: The search did not return the full conversation with Alice from Anthropic. I will try searching for another researcher's conversation....
Thought: It seems there are no email conversations found with the researchers invited based on the initial search results. To get more information, I will try a broader search for emails related to "paper reading"....
Thought: The new search results show the initial invitations sent to Alice, Jane, and Bob to present their papers at the r

In [80]:
prompt = "What was the name of the guy from Nexus Dynamic Solutions that sent me a couple of emails?"
generate_response(prompt)

Thought: To find the name of the person from Nexus Dynamic Solutions who sent emails, I should first search for emails mentioning the company to see if any conversations took place. If I find a relevant email, I can then use the find_conversations tool to retrieve the full email thread and look for the person's name....
Thought: The search results show that there were a couple of emails from Nexus Dynamic Solutions, specifically from someone named Alex. To get Alex's full name and confirm he was the only one who emailed me from NDS, I should find the full conversation thread with him....
Thought: Based on the full conversation thread, it looks like Alex Nexus was the only person from Nexus Dynamic Solutions who emailed me. He sent a couple of welcome emails after I signed up for their service. I have enough information to provide a final answer to the original question....
Answer:  The person from Nexus Dynamic Solutions who sent you a couple of emails is Alex Nexus. He reached out to 

# Limitations of Agentic RAG

Getting LLMs to adhere to the system prompt's instructions is extremely challenging. It requires meticulous fine-tuning of the system prompt through testing numerous examples. Furthermore, a system prompt that is effective with one model may not necessarily be effective with others. Let's explore this now!

In [81]:
unify_client.set_endpoint("llama-2-70b-chat@together-ai")

In [82]:
prompt = "What was the name of the guy from Nexus Dynamic Solutions that sent me a couple of emails?"
generate_response(prompt)

BadRequestError: Error code: 400 - {'detail': '<!DOCTYPE html>\n<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]-->\n<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]-->\n<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en-US"> <![endif]-->\n<!--[if gt IE 8]><!--> <html class="no-js" lang="en-US"> <!--<![endif]-->\n<head>\n\n\n<title>api.together.xyz | 524: A timeout occurred</title>\n<meta charset="UTF-8" />\n<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />\n<meta http-equiv="X-UA-Compatible" content="IE=Edge" />\n<meta name="robots" content="noindex, nofollow" />\n<meta name="viewport" content="width=device-width,initial-scale=1" />\n<link rel="stylesheet" id="cf_styles-css" href="/cdn-cgi/styles/main.css" />\n\n\n</head>\n<body>\n<div id="cf-wrapper">\n    <div id="cf-error-details" class="p-0">\n        <header class="mx-auto pt-10 lg:pt-6 lg:px-8 w-240 lg:w-full mb-8">\n            <h1 class="inline-block sm:bl

The failure occurred because the response from `llama-2-70b-chat` did not include the string "Action Input", despite the system prompt's explicit instruction to always contain the function arguments after "Action Input" string. It's important to clarify that this doesn't necessarily indicate `llama-2-70b-chat`'s inability to adhere to the system prompt. Instead, it suggests that the system prompt may require adjustments to be compatible with it. Let's try with some other models.

In [62]:
unify_client.set_endpoint("mixtral-8x7b-instruct-v0.1@together-ai")

In [63]:
prompt = "Who are the researchers (and their papers) that accepted and declined my Paper reading invitation?"
generate_response(prompt)

Thought: To answer this question, I need to first find all the emails related to the Paper reading invitation. I will use the 'search_query' tool to fetch relevant emails. After that, I'll look into those emails to find out the researchers who accepted and declined the invitation. Once I have their email addresses, I can use the 'find_conversations' tool to get the full context of each conversation, which should include the researchers' papers....
Thought: The search results show four email threads related to the Paper reading invitation - one from John Doe accepting the invitation, and three invitations sent to Alice, Bob, and Jane. I need to find the responses from Alice, Bob, and Jane to determine whether they accepted or declined the invitation. I will once again use the 'search_query' tool to look for their responses specifically....
Thought: The search results only show the original invitations for Alice, Bob, and Jane, but not their responses. I will use the 'find_conversations'

JSONDecodeError: Expecting value: line 3 column 16 (char 52)

Here the failure occured because Mixtral didn't generate the function arguments in the format we had specified.

#### To summarise
Adhering to system prompt instructions is challenging for large language models (LLMs) and requires meticulous fine-tuning. Key observations include:

* **Model-Specific Prompting:** Effective prompts for one model may not work for another, necessitating model-specific adjustments.
* **Prompt Refinement:** The failure of llama-2-70b-chat to include "Action Input" suggests the need for further prompt refinement for compatibility.
* **Format Adherence:** Mixtral's failure to generate the specified function arguments highlights the importance of precise prompt structuring.

These findings underscore the complexity of creating universal prompts and the need for adaptable prompting strategies to ensure consistent model responses.